This example shows a basic example of "Feff fitting" with larch.  

We start with some import statements, then 
   1. read in data for ZnSe, Zn K edge
   2. do background subtraction and plot $\chi(k)$
   3. set the Fourier transform ranges
   4. define a Group of Parameters for the fit
   5. read in a Feff Path for Zn-Se 
   6. run the fit and view the results

In [1]:
# step 0: imports
import numpy as np
from larch.xafs import (pre_edge, autobk, xftf, 
                        ff2chi, feffpath, feffit_transform, feffit_dataset, feffit, feffit_report)
from larch.fitting import param, guess, param_group
from larch.io import read_ascii
from larch.plot.bokeh_xafsplots import (plot_mu, plot_chik, plot_chir, plot_chifit, 
                                         multi_plot, plotlabels, set_label_weight)

In [2]:
# step 1: read in the data and make mu(E)

data  = read_ascii('../xafsdata/znse_zn_xafs.001', labels=['energy', 'time', 'i0', 'itrans'])
data.mu = -np.log(data.itrans/data.i0)
pre_edge(data)
fig1 = plot_mu(data, show_norm=True)

Loading BokehJS ...

In [3]:
# step 2: do and check the background subtraction
autobk(data, rbkg=1.0, kw=2)
fig2 = plot_chik(data, kweight=2)

Loading BokehJS ...

In [4]:
# step 3: set transform / fit ranges
trans = feffit_transform(kmin=3, kmax=12, kweight=2, dk=4, window='kaiser', rmin=1.0, rmax=3.0)
xftf(data, kmin=3, kmax=12, kweight=2, dk=4, window='kaiser')
fig3 = plot_chir(data)

Loading BokehJS ...

In [5]:
# step 4: setup fitting Parameters: we will vary 
#   amplitude, e0, sigma2, and delta_R

pars = param_group(amp    = param(1.0, vary=True),
                   del_e0 = param(0.5, vary=True),
                   sig2   = param(0.005, vary=True),
                   del_r  = guess(0.001, vary=True))

In [6]:
# step 5: define a Feff Path for Zn-Se scattering

# using a Path file from a previous Feff calculation for Zn-Se (shown elsewhere)
# here we give math expressions using the Parameters named in the Parameter group 
# to set the values for the Path Parameters in the EXAFS equation
path_znse = feffpath('../feffit/feff_znse.dat', 
                     s02='amp', e0='del_e0', sigma2='sig2', deltar='del_r')

tmpsum = ff2chi([path_znse], params=pars)
fig4 = plot_chik(tmpsum, kweight=2)
fit5 = multi_plot([{'xdata':tmpsum.k, 'ydata':tmpsum.chi*tmpsum.k**2, 'label': 'Feff'},
            {'xdata':data.k, 'ydata':data.chi*data.k**2, 
             'label': 'Data','xlabel': plotlabels.k, 
             'ylabel': set_label_weight(plotlabels.chikw, 2) }])


Loading BokehJS ...

Loading BokehJS ...

In [7]:
# step 6: run and view the fit

# now that we have data, a set of paths, and Fourier transform values, we can
# set up a "Feffit Dataset" and then fit that with the group of Parameters

# define dataset to include data, pathlist, transform
dset = feffit_dataset(data=data, pathlist=[path_znse], transform=trans)

# do the fit...
result = feffit(pars, dset)

# print out the fit report
print(feffit_report(result))

=================== FEFFIT RESULTS ====================
[[Statistics]]
  n_function_calls     = 26
  n_variables          = 4
  n_data_points        = 130
  n_independent        = 12.4591559
  chi_square           = 246.372951
  reduced chi_square   = 29.1250042
  r-factor             = 0.00283002
  Akaike info crit     = 45.1829888
  Bayesian info crit   = 47.2728119
 
[[Parameters]]
  amp                  =  0.9907808 +/- 0.0545694  (init= 1.0000000)
  del_e0               =  5.8934099 +/- 0.6999559  (init= 0.5000000)
  del_r                = -0.0012321 +/- 0.0034249  (init= 1.0000e-3)
  sig2                 =  0.0065432 +/- 4.1835e-4  (init= 0.0050000)
 
[[Correlations]] (unreported correlations are <  0.100)
  amp, sig2            =  0.943
  del_e0, del_r        =  0.932
  amp, del_r           =  0.196
  del_r, sig2          =  0.191
 
[[Dataset]]
  unique_id            = 'dudpz4dw'
  fit space            = 'r'
  r-range              = 1.000, 3.000
  k-range              = 3.000, 1

In [8]:
# finally, plot the results
fig6, fig7 = plot_chifit(result.datasets[0], show_real=True)

Loading BokehJS ...

Loading BokehJS ...